# Setup

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import scipy
import abc
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from keras.optimizers import Adam
from keras.applications import VGG16


Using TensorFlow backend.


In [2]:
train = pd.read_json("_RawData/train.json/data/processed/train.json")
test = pd.read_json("_RawData/test.json/data/processed/test.json")

# CNN

In [3]:
def get_channel(band_data):
    print("get_channel")
    band_data = np.array([np.array(band).astype(np.float32) for band in band_data])
    
    print(band_data.shape)
    band_data_new = np.empty((1604, 150, 150))

    for ind in range(band_data.shape[0]):
        arr_image = band_data[ind]
        band_data_new[ind] = (scipy.misc.fromimage(scipy.misc.toimage(arr_image.reshape(75, 75)).resize((150, 150))))

    print(band_data_new.shape)

    return band_data_new

def concat_data(data):
    channel1 = get_channel(data["band_1"])
    channel2 = get_channel(data["band_2"])
    channel3 = (channel1 + channel2) / 2.

    return np.concatenate([channel1[:, :, :, np.newaxis], channel2[:, :, :, np.newaxis],channel3[:, :, :, np.newaxis]], axis=-1)

In [4]:
X_train = concat_data(train)

get_channel
(1604, 5625)


C:\Users\Dave\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Dave\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `fromimage` is deprecated!
`fromimage` is deprecated in SciPy 1.0.0. and will be removed in 1.2.0.
Use ``np.asarray(im)`` instead.
  # Remove the CWD from sys.path while we load stuff.


(1604, 150, 150)
get_channel
(1604, 5625)
(1604, 150, 150)


In [5]:
from sklearn.model_selection import train_test_split

y_train=train['is_iceberg']
X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, y_train, random_state=1, train_size=0.75)

C:\Users\Dave\Anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\model_selection\_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [6]:
class DaveBaseModel:
    __metaclass__ = abc.ABCMeta
    
    def __init__(self, X_train, X_valid, y_train, y_valid):
        self.X_train = X_train
        self.y_train = y_train
        self.X_valid = X_valid
        self.y_valid = y_valid
        self.dgen, self.vgen = DaveBaseModel.get_generators(X_train, X_valid)
        
        self.get_model()
        
    def train(self, epochs, batch_size):
        self.model.fit_generator(self.dgen.flow(self.X_train, self.y_train, batch_size=batch_size),
                            steps_per_epoch=len(self.X_train) / batch_size, 
                            validation_data=self.vgen.flow(self.X_valid, self.y_valid, batch_size=batch_size, shuffle=False),
                            validation_steps = len(self.X_valid) / batch_size,
                            epochs=epochs)

    def get_generators(train_data, valid_data):
        data_gen = ImageDataGenerator(
                rescale=1./255,
                shear_range=0.2,
                zoom_range=0.2,
                rotation_range=180,
                vertical_flip=True,
                horizontal_flip=True)

        data_gen.fit(train_data)

        val_gen = ImageDataGenerator(rescale=1./255)
        val_gen.fit(valid_data)

        return data_gen, val_gen
    
    @abc.abstractmethod
    def get_model():
        return
    

class DaveVGG(DaveBaseModel):
    def get_model(self):
        vgg_model = VGG16(include_top=False, weights=None, input_shape=(150, 150, 3))

        top_model = Sequential()
        top_model.add(Flatten(input_shape=vgg_model.output_shape[1:]))
        top_model.add(Dense(512, activation='relu'))
        top_model.add(Dense(512, activation='relu'))
        top_model.add(Dropout(0.5))
        top_model.add(Dense(1, activation='sigmoid'))
        # top_model.load_weights(top_model_weights_path)

        self.model = Model(inputs= vgg_model.input, outputs= top_model(vgg_model.output))

        self.model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])
        print(self.model.summary())


In [7]:
model = DaveVGG(X_train_cv, X_valid, y_train_cv, y_valid)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [31]:
model.train(20, 32)

Epoch 1/20
38/37 [==============================] - 15s - loss: 0.4200 - acc: 0.8067 - val_loss: 0.3860 - val_acc: 0.8155
Epoch 2/20
38/37 [==============================] - 15s - loss: 0.3984 - acc: 0.8185 - val_loss: 0.3744 - val_acc: 0.8504
Epoch 3/20
38/37 [==============================] - 15s - loss: 0.4189 - acc: 0.8141 - val_loss: 0.3942 - val_acc: 0.8105
Epoch 4/20
38/37 [==============================] - 15s - loss: 0.4080 - acc: 0.8218 - val_loss: 0.3652 - val_acc: 0.8404
Epoch 5/20
38/37 [==============================] - 15s - loss: 0.3715 - acc: 0.8316 - val_loss: 0.3827 - val_acc: 0.8379
Epoch 6/20
38/37 [==============================] - 15s - loss: 0.3950 - acc: 0.8174 - val_loss: 0.3484 - val_acc: 0.8579
Epoch 7/20
38/37 [==============================] - 15s - loss: 0.3652 - acc: 0.8366 - val_loss: 0.3368 - val_acc: 0.8653
Epoch 8/20
38/37 [==============================] - 16s - loss: 0.3673 - acc: 0.8432 - val_loss: 0.3526 - val_acc: 0.8529
Epoch 9/20
38/37 [======

In [8]:
from keras.applications import InceptionV3

class DaveInceptionV3(DaveBaseModel):
    def get_model(self):
        v3_model = InceptionV3(include_top=False, weights=None, input_shape=(150, 150, 3))

        x = v3_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(512, activation='relu')(x)
        x = Dropout(0.5)(x)
        x = Dense(512, activation='relu')(x)
        x = Dropout(0.5)(x)
        predictions = Dense(1, activation='sigmoid')(x)
        
        self.model = Model(inputs=v3_model.input, outputs=predictions)

        self.model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])
        print(self.model.summary())

In [9]:
model = DaveInceptionV3(X_train_cv, X_valid, y_train_cv, y_valid)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 74, 74, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 74, 74, 32)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 74, 74, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [11]:
model.train(10, 32)

Epoch 1/10
38/37 [==============================] - 11s 280ms/step - loss: 0.3747 - acc: 0.8092 - val_loss: 3.2590 - val_acc: 0.5312
Epoch 2/10
38/37 [==============================] - 10s 274ms/step - loss: 0.3744 - acc: 0.8297 - val_loss: 2.7645 - val_acc: 0.5312
Epoch 3/10
38/37 [==============================] - 10s 274ms/step - loss: 0.3625 - acc: 0.8316 - val_loss: 2.6170 - val_acc: 0.5337
Epoch 4/10
38/37 [==============================] - 10s 275ms/step - loss: 0.3674 - acc: 0.8284 - val_loss: 1.5196 - val_acc: 0.5511
Epoch 5/10
38/37 [==============================] - 11s 277ms/step - loss: 0.3553 - acc: 0.8347 - val_loss: 1.4838 - val_acc: 0.5511
Epoch 6/10
38/37 [==============================] - 10s 276ms/step - loss: 0.3542 - acc: 0.8454 - val_loss: 0.5735 - val_acc: 0.7307
Epoch 7/10
38/37 [==============================] - 10s 275ms/step - loss: 0.3490 - acc: 0.8305 - val_loss: 0.7565 - val_acc: 0.6858
Epoch 8/10
38/37 [==============================] - 10s 274ms/step - 